In [ ]:
!nvidia-smi

Mon May  3 17:30:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pickle

import torch
!pip install transformers
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm.notebook import tqdm
from scipy import stats
import collections
import sklearn
import re

from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
with open("/content/drive/My Drive/fakes_plus_real7000.pickle", 'rb') as f:
    data = pickle.load(f)

len(data)

6902

In [ ]:
str = data[list(data.keys())[2]]['real_comment']
print(str)
str = re.sub(r'\n', ' ', str)
str

i saw the new spiderman recently and i think it's the best spiderman film made so far. i also saw 21 and over like last week and it was pretty funny.


"i saw the new spiderman recently and i think it's the best spiderman film made so far. i also saw 21 and over like last week and it was pretty funny."

In [ ]:
# load sample data
comments = []
labels = []
for key in data.keys():
  real = data[key]['real_comment']
  if len(real) <= 500:
    comments.append(real)
    labels.append(1)

  fake = data[key]['fake_comment']
  if len(fake) <= 500:
    comments.append(fake)
    labels.append(0)


In [ ]:
len(comments)

13315

In [ ]:
# split text
train_texts, test_texts, train_labels, test_labels = train_test_split(comments, labels, test_size=.1, train_size=.9, shuffle=True)
# train_texts, test_texts, train_labels, test_labels = train_test_split(posts_comms, upvotes, test_size=.1, train_size=.2, shuffle=True)
print(len(train_texts), len(train_labels), len(test_texts), len(test_labels))

posts_comms = None

11983 11983 1332 1332


In [ ]:
# load tokenizer
tokenizer = BertTokenizerFast.from_pretrained('distilbert-base-cased', use_cache=True)

In [ ]:
# tokenize train data
train_encodings = tokenizer(text=train_texts, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
# tokenize test data
test_encodings = tokenizer(text=test_texts, truncation=True, padding=True)

In [ ]:
# load model
model = BertForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=2)
model.train()

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of the model checkpoint at distilbert-base-cased were not used when initializing BertForSequenceClassification: ['distilbert.embeddings.word_embeddings.weight', 'distilbert.embeddings.position_embeddings.weight', 'distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias', 'distilbert.transformer.layer.0.attention.q_lin.weight', 'distilbert.transformer.layer.0.attention.q_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.weight', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.0.attention.v_lin.weight', 'distilbert.transformer.layer.0.attention.v_lin.bias', 'distilbert.transformer.layer.0.attention.out_lin.weight', 'distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.sa_layer_norm.weight', 'distilbert.transformer.layer.0.sa_layer_norm.bias', 'distilbert.transformer.layer.0.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin1.bias', 'distilbert.transformer.layer.0.ff

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# create dataset class and load encodings and associated labels to it

class upvote_prediction_dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = upvote_prediction_dataset(train_encodings, train_labels)
test_dataset = upvote_prediction_dataset(test_encodings, test_labels)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()

Step,Training Loss
10,0.711500
20,0.699000
30,0.688200
40,0.705300
50,0.674100
60,0.693700
70,0.703500
80,0.678100
90,0.690100
100,0.639100


TrainOutput(global_step=3745, training_loss=0.36139661296505793, metrics={'train_runtime': 1678.8893, 'train_samples_per_second': 2.231, 'total_flos': 8059961603418300.0, 'epoch': 5.0, 'init_mem_cpu_alloc_delta': 1874640896, 'init_mem_gpu_alloc_delta': 433776128, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 17035264, 'train_mem_gpu_alloc_delta': 1321523712, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 3455978496})

In [ ]:
trainer.save_model('/content/drive/My Drive/realism_discriminator')

In [ ]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [ ]:
# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print("You are using device: %s" % device)

You are using device: cuda


In [ ]:
# load model
model = BertForSequenceClassification.from_pretrained('/content/drive/My Drive/realism_discriminator', num_labels=2)
model.eval()

model.to(device)
items = {key: torch.tensor(val).to(device) for key, val in test_encodings.items()}

In [ ]:
preds = []
for batch_idxs in batch(range(len(items['input_ids'])), 16):
  batch_input_ids = items['input_ids'][batch_idxs]
  batch_attention_mask = items['attention_mask'][batch_idxs]
  batch_token_type_ids = items['token_type_ids'][batch_idxs]

  outs = model.forward(input_ids=batch_input_ids, attention_mask=batch_attention_mask, token_type_ids=batch_token_type_ids)
  probs = torch.softmax(outs.logits, 1)
  classes = torch.argmax(probs, dim=1)

  preds += list(classes.cpu().numpy())

In [ ]:
sklearn.metrics.f1_score(preds, test_labels, average='weighted')

0.9421676109984425

In [ ]:
sklearn.metrics.confusion_matrix(preds, test_labels)

array([[702,  36],
       [ 41, 553]])